# Running a Grid

A common task is to run UCLCHEM over a grid of parameter combinations. This notebook sets up a simple approach to doing so for regular grids.

In [1]:
import uclchem
import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed

## A Simple Grid
### Define Parameter Space
First, we define our parameter space. We do this by using numpy and pandas to produce a table of all possible combinations of some parameters of interest.

In [2]:
# This part can be substituted with any choice of grid
# here we just vary the density, temperature and zeta
temperatures = np.linspace(10, 50, 3)
densities = np.logspace(4, 6, 3)
zetas = np.logspace(1, 3, 3)

# meshgrid will give all combinations, then we shape into columns and put into a table
parameterSpace = np.asarray(np.meshgrid(temperatures, densities, zetas)).reshape(3, -1)
model_table = pd.DataFrame(parameterSpace.T, columns=["temperature", "density", "zeta"])

# keep track of where each model output will be saved and make sure that folder exists
model_table["outputFile"] = model_table.apply(
    lambda row: f"../grid_folder/{row.temperature}_{row.density}_{row.zeta}.csv", axis=1
)
print(f"{model_table.shape[0]} models to run")
if not os.path.exists("../grid_folder"):
    os.makedirs("../grid_folder")

27 models to run


In [3]:
model_table.head()

,temperature,density,zeta,outputFile
0,10.0,10000.0,10.0,../grid_folder/10.0_10000.0_10.0.csv
1,10.0,10000.0,100.0,../grid_folder/10.0_10000.0_100.0.csv
2,10.0,10000.0,1000.0,../grid_folder/10.0_10000.0_1000.0.csv
3,30.0,10000.0,10.0,../grid_folder/30.0_10000.0_10.0.csv
4,30.0,10000.0,100.0,../grid_folder/30.0_10000.0_100.0.csv


### Set up the model
Next, we need a function that will run our model. We write a quick function that takes a row from our dataframe and uses it to populate a parameter dictionary for UCLCHEM and then run a cloud model. We can then map our dataframe to that function.

In [4]:
def run_model(row):
    # basic set of parameters we'll use for this grid.
    ParameterDictionary = {
        "endatfinaldensity": False,
        "freefall": False,
        "initialDens": row.density,
        "initialTemp": row.temperature,
        "zeta": row.zeta,
        "outputFile": row.outputFile,
        "finalTime": 1.0e6,
        "baseAv": 10,
        "abstol_min": 1e-22,
    }
    result = uclchem.model.cloud(param_dict=ParameterDictionary)
    return result[0]  # just the integer error code

### Run Grid 

#### The Simple Way
We can use pandas apply to simply pass each row to our helper function in turn. This will take some time since we're running the models one by one. I'll use the `head` function just to run five rows as an example here.

In [5]:
%%time
result = model_table.head().apply(run_model, axis=1)

CPU times: user 26.1 s, sys: 328 ms, total: 26.4 s
Wall time: 26.4 s


#### The Fast Way
Alternatively, we can use multiprocessing to run the models in parallel. That will allow us to run many models simulataneously and make use of all the cores available on our machine.

In [6]:
%%time
results = Parallel(n_jobs=4, verbose=100)(
    delayed(run_model)(row) for idx, row in model_table.iterrows()
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    2.4s


[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    3.2s


[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    3.5s


[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    4.6s


...

 At current T(=R1), MXSTEP(=I1) steps                                            
 taken on this call before reaching TOUT.                                        
In the above message, I1 =      10000
In the above message, R1 =   0.2917054938328D+14
 ISTATE -1: Reducing time step to    7688.1349775940917      years


 At current T(=R1), MXSTEP(=I1) steps                                            
 taken on this call before reaching TOUT.                                        
In the above message, I1 =      10000
In the above message, R1 =   0.2996640926682D+14
 ISTATE -1: Reducing time step to    5169.5910048165933      years


 At current T(=R1), MXSTEP(=I1) steps                                            
 taken on this call before reaching TOUT.                                        
In the above message, I1 =      10000
In the above message, R1 =   0.3072871858243D+14
 ISTATE -1: Reducing time step to    2757.2197169288575      years


 At current T(=R1), MXSTEP(=I1) steps                                            
 taken on this call before reaching TOUT.                                        
In the above message, I1 =      10000
In the above message, R1 =   0.3137412072161D+14
 ISTATE -1: Reducing time step to    714.80785365212216      years


[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed: 21.0min finished
CPU times: user 355 ms, sys: 405 ms, total: 760 ms
Wall time: 20min 58s


## Checking Your Grid
After running, we should do two things. First, let's add `results` to our dataframe as a new column. Positive results mean a successful UCLCHEM run and negative ones are unsuccessful. Then we can run each model through `check_element_conservation` to check the integration was successful. We'll use both these things to flag models that failed in some way.

In [7]:
def element_check(output_file):
    df = uclchem.analysis.read_output_file(output_file)
    # get conservation values
    conserves = uclchem.analysis.check_element_conservation(df)
    # check if any error is greater than 1%
    return all([float(x[:-1]) < 1 for x in conserves.values()])

In [8]:
model_table["run_result"] = results
model_table["elements_conserved"] = model_table["outputFile"].map(element_check)
# check both conditions are met
model_table["Successful"] = (model_table.run_result >= 0) & (
    model_table.elements_conserved
)

In [9]:
model_table.head()

,temperature,density,zeta,outputFile,run_result,elements_conserved,Successful
0,10.0,10000.0,10.0,../grid_folder/10.0_10000.0_10.0.csv,0,True,True
1,10.0,10000.0,100.0,../grid_folder/10.0_10000.0_100.0.csv,0,True,True
2,10.0,10000.0,1000.0,../grid_folder/10.0_10000.0_1000.0.csv,0,True,True
3,30.0,10000.0,10.0,../grid_folder/30.0_10000.0_10.0.csv,0,True,True
4,30.0,10000.0,100.0,../grid_folder/30.0_10000.0_100.0.csv,0,True,True


## Complex Grid

The above was straightforward enough but what about a modelling a grid of shocks? Not only do we want to loop over relevant parameters, we also need to run a few preliminary models to give ourselves starting abundances. We'll start by defining two helper functions, one to run our preliminary cloud and one to run the shock.

Let's further imagine that we want to obtain the abundances of several species at the end of the model. We can use the `out_species` parameter to specify which species we want to track and return them to our dataframe.

In [10]:
out_species = ["CO", "H2O", "CH3OH"]


def run_prelim(density):
    # basic set of parameters we'll use for this grid.
    ParameterDictionary = {
        "endatfinaldensity": True,
        "freefall": True,
        "initialDens": 1e2,
        "finalDens": float(density),
        "initialTemp": 10.0,
        "abundSaveFile": f"../grid_folder/starts/{density:.0f}.csv",
        "baseAv": 1,
    }
    result = uclchem.model.cloud(param_dict=ParameterDictionary)
    return result


def run_model(row):
    # basic set of parameters we'll use for this grid.
    ParameterDictionary = {
        "endatfinaldensity": False,
        "freefall": False,
        "initialDens": float(row.density),
        "initialTemp": 10.0,
        "outputFile": row.outputFile,
        "abundLoadFile": f"../grid_folder/starts/{row.density:.0f}.csv",
        "finalTime": 1.0e5,
        "abstol_factor": 1e-14,
        "abstol_min": 1e-20,
        "reltol": 1e-6,
        "baseAv": 1,
    }
    result = uclchem.model.cshock(
        row.shock_velocity, param_dict=ParameterDictionary, out_species=out_species
    )
    # First check UCLCHEM's result flag to seeif it ran succesfully, if it is return the abundances
    if result[0] == 0:
        return result[:]
    # if not, return NaNs because model failed
    else:
        return [np.nan] * len(out_species)

Then we define our parameter space again. We'll create two folders, one to store a set of initial abundances for each starting density in our model and another to store our shock outputs.

In [11]:
# This part can be substituted with any choice of grid
# here we just combine various initial and final densities into an easily iterable array
shock_velocities = np.linspace(10, 50, 3)
densities = np.logspace(4, 6, 3)

parameterSpace = np.asarray(np.meshgrid(shock_velocities, densities)).reshape(2, -1)
model_table = pd.DataFrame(parameterSpace.T, columns=["shock_velocity", "density"])
model_table["outputFile"] = model_table.apply(
    lambda row: f"../grid_folder/shocks/{row.shock_velocity}_{row.density}.csv", axis=1
)
print(f"{model_table.shape[0]} models to run")

for folder in ["starts", "shocks"]:
    if not os.path.exists(f"../grid_folder/{folder}"):
        os.makedirs(f"../grid_folder/{folder}")

9 models to run


We can then run our preliminary models followed by our science models. The science models will return the abundances at the final time step of each run so we can unpack those directly to our dataframe.

In [12]:
results = Parallel(n_jobs=4, verbose=100)(
    delayed(run_prelim)(dens) for dens in densities
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    4.8s


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:    8.2s finished


In [13]:
results = Parallel(n_jobs=4, verbose=100)(
    delayed(run_model)(row) for idx, row in model_table.iterrows()
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   10.0s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   10.1s


[Parallel(n_jobs=4)]: Done   3 out of   9 | elapsed:   17.6s remaining:   35.1s


[Parallel(n_jobs=4)]: Done   4 out of   9 | elapsed:   21.2s remaining:   26.4s


 At current T(=R1), MXSTEP(=I1) steps                                            
 taken on this call before reaching TOUT.                                        
In the above message, I1 =      10000
In the above message, R1 =   0.3063046680601D+12
 ISTATE -1: Reducing time step to    96.771568540453785      years


...

 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.3041993188779D+13   R2 =   0.1781774460333D+02
 ISTATE -5 - shortening step at time   95725.159790072867      years


 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.3042001498960D+13   R2 =   0.1312846642968D-04
 ISTATE -5 - shortening step at time   95725.159790072867      years


 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.3042064195111D+13   R2 =   0.4702814028576D-05
 ISTATE -5 - shortening step at time   95725.159790072867      years
 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.3042073427456D+13   R2 =   0.9911190692136D+01
 ISTATE -5 - shortening step at time   95725.159790072867      years


 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.3042077414578D+13   R2 =   0.1209916820412D+02
 ISTATE -5 - shortening step at time   95725.159790072867      years
 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.3042093081334D+13   R2 =   0.2109333346823D+02
 ISTATE -5 - shortening step at time   95725.159790072867      years


 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.3042099653302D+13   R2 =   0.1023113238489D+02
 ISTATE -5 - shortening step at time   95725.159790072867      years
[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed: 28.7min finished


In [14]:
model_table[["Result", "Dissipation Time"] + out_species] = results

In [15]:
model_table

,shock_velocity,density,outputFile,Result,Dissipation Time,CO,H2O,CH3OH
0,10.0,10000.0,../grid_folder/shocks/10.0_10000.0.csv,0.0,1171.898734,7.727502e-05,9.032445e-11,1.091258e-13
1,30.0,10000.0,../grid_folder/shocks/30.0_10000.0.csv,0.0,1171.898734,3.365962e-05,1.190018e-05,6.000710e-14
2,50.0,10000.0,../grid_folder/shocks/50.0_10000.0.csv,0.0,1171.898734,1.764835e-05,8.205893e-06,2.386004e-09
3,10.0,100000.0,../grid_folder/shocks/10.0_100000.0.csv,0.0,117.189873,7.202338e-11,3.043583e-11,4.026625e-14
4,30.0,100000.0,../grid_folder/shocks/30.0_100000.0.csv,0.0,117.189873,3.111801e-24,3.403524e-22,3.227464e-22
5,50.0,100000.0,../grid_folder/shocks/50.0_100000.0.csv,0.0,117.189873,2.108334e-24,2.208829e-22,2.307490e-22
6,10.0,1000000.0,../grid_folder/shocks/10.0_1000000.0.csv,0.0,11.718987,3.586509e-27,2.642741e-23,5.058576e-23
7,30.0,1000000.0,../grid_folder/shocks/30.0_1000000.0.csv,0.0,11.718987,4.718451e-25,1.063721e-20,1.000000e-30
8,50.0,1000000.0,../grid_folder/shocks/50.0_1000000.0.csv,0.0,11.718987,1.000000e-30,4.127471e-23,2.321015e-23


## Summary

There are many ways to run grids of models and users will naturally develop their own methods. This notebook is just a simple example of how to run UCLCHEM for many parameter combinations whilst producing a useful output (the model_table) to keep track of all the combinations that were run. In a real script, we'd save the model file to csv at the end.

For much larger grids, it's recommended that you find a way to make your script robust to failure. Over a huge range of parameters, it is quite likely UCLCHEM will hit integration trouble for at least a few parameter combinations. Very occasionally, UCLCHEM will get caught in a loop where it fails to integrate and cannot adjust its strategy to manage it. This isn't a problem for small grids as the model can be stopped and the tolerances adjusted. However, for very large grids, you may end up locking all threads as they each get stuck on a different model. The best solution we've found for this case is to add a check so that models in your dataframe are skipped if their file already exists, this allows you to stop and restart the grid script as needed.
